In [1]:
using Pkg
pkg"activate ."

In [154]:
using Test
using BenchmarkTools

The chart consists of a list of arcs, some of which are completed. Each arc is an edge from one point in the sentence to another, labeled by the production rule it corresponds to. 

This isn't mentioned in the presentations I've read so far, but I assume that each edge also should contain a pointer to the edges that were found when constructing it, in order to be able to recover the full parse later. 

In the cryptics problem, there's an additional complication: a given production rule can produce many outputs, and we need to keep track of each of them. One way to do that would be to create multiple edges, each labled not only with the production rule but also the resulting output word. That seems like a natural approach when we combine this with some probabilistic notion, as the quality of the output word will likely influence the probability of the resulting parse. 



In [2]:
module Charts

struct Edge
    head::Symbol
    arguments::Vector{Edge}
    output::String
end

end


Main.Charts

We can either store the endpoints of each edge in the Edge type itself, or we could store the edges based on their endpoints (for example, as a matrix of vectors of edges). I don't think edges need to be mutated, so we wouldn't have to move edges from one list to another, only add new ones. So either representation could be efficient. 

What are the operations we need to do?

I think that maybe I should implement a very basic chart parser first, without worrying about output words, in order to get a sense of how this stuff works. 


In [3]:
module Charts

struct Edge
    head::Symbol
    components::Vector{Symbol}
    completed::Int
    span::Pair{Int}
end

# In the fundamental rule, we need to look up completed arcs
# by their first span index and their head type
# In the bottom-up rule, we look through completed arcs to 
# try to match them with rules in the grammar. 

struct Chart
    active_arcs::Vector{Edge}
    agenda::Vector{Edge}
    inactive_arcs::Matrix{Vector{Edge}}
end

function Chart(len::Integer)
    Chart(
        Edge[],
        Edge[],
        [edges for i in 1:len, j in 1:len])
end

# function fundamental_rule!(chart::Chart)
#     for arc in chart.active_arcs
#         i, j = arc.span
        
# end

end


Main.Charts

Let's try directly following the example, from http://cs.union.edu/~striegnk/courses/nlp-with-prolog/html/node71.html#l9.sec.bottomup

In [35]:
module Charts

struct Edge
    head::Symbol
    constituents::Vector{Symbol}
    start::Int
    stop::Int
    completed::Int
end

function Base.hash(e::Edge, h::UInt)
    h = hash(e.head, h)
    h = hash(e.constituents, h)
    h = hash(e.start, h)
    h = hash(e.stop, h)
    h = hash(e.completed, h)
end

function Base.:(==)(e1::Edge, e2::Edge)
    e1.head == e2.head &&
    e1.constituents == e2.constituents &&
    e1.start == e2.start &&
    e1.stop == e2.stop &&
    e1.completed == e2.completed
end

isactive(edge::Edge) = edge.completed < length(edge.constituents)
needs(edge::Edge) = edge.constituents[edge.completed + 1]

function Base.show(io::IO, e::Edge)
    constituents = copy(e.constituents)
    insert!(constituents, e.completed + 1, :.)
    print(io, "<$(e.start), $(e.stop), $(e.head) -> $(join(constituents, ' '))>")
end

function parse()
    grammar = [
        :S => [:NP, :VP, :PP],
        :S => [:NP, :VP],
        :NP => [:PN],
        :VP => [:IV],
        :PP => [:P, :NP],
    ]
    
    # step 1
    agenda = Edge[]
    chart = Edge[]
    
    push!(agenda, Edge(:PN, [:mia], 0, 1, 1))
    push!(agenda, Edge(:IV, [:danced], 1, 2, 1))
    
    while !isempty(agenda)
#         println("beginning:")
#         @show chart agenda
        
        # step 2a
        edge = popfirst!(agenda)
#         @show edge
        
        # step 2b
        if edge ∉ chart
            println("moving: $edge to chart")
            push!(chart, edge)
        end
        
        # step 2c
        if isactive(edge)
            s = needs(edge)
            for arc in chart
                if !isactive(arc) && arc.start == edge.stop && arc.head == s
                    combined = Edge(edge.head, edge.constituents, edge.start, arc.stop, edge.completed + 1)
                    if combined ∉ chart
                        println("fundamental rule: ", combined)
                        pushfirst!(agenda, combined)
                    end
                end
            end
        else
            for arc in chart
                if isactive(arc) && arc.stop == edge.start && needs(arc) == edge.head
                    combined = Edge(arc.head, arc.constituents, arc.start, edge.stop, arc.completed + 1)
                    if combined ∉ chart
                        println("fundamental rule: ", combined)
                        pushfirst!(agenda, combined)
                    end
                end
            end
        end
        
        # step 2d
        if !isactive(edge)
            for rule in grammar
                if rule[2][1] == edge.head
                    candidate = Edge(rule[1], rule[2], edge.start, edge.start, 0)
                    if candidate ∉ chart
                        println("bottom-up: ", candidate)
                        pushfirst!(agenda, candidate)
                    end
                        
                end 
            end
        end
        
#         println("end")
        @show chart agenda
        isempty(readline(stdin)) || break
    end
end


end

Main.Charts

In [187]:
++(x, y) = "$x $y"

++ (generic function with 1 method)

In [188]:
1 ++ 2

"1 2"

In [36]:
Charts.parse()

moving: <0, 1, PN -> mia .> to chart
bottom-up: <0, 0, NP -> . PN>
chart = Main.Charts.Edge[<0, 1, PN -> mia .>]
agenda = Main.Charts.Edge[<0, 0, NP -> . PN>, <1, 2, IV -> danced .>]
stdin> 
moving: <0, 0, NP -> . PN> to chart
fundamental rule: <0, 1, NP -> PN .>
chart = Main.Charts.Edge[<0, 1, PN -> mia .>, <0, 0, NP -> . PN>]
agenda = Main.Charts.Edge[<0, 1, NP -> PN .>, <1, 2, IV -> danced .>]
stdin> 
moving: <0, 1, NP -> PN .> to chart
bottom-up: <0, 0, S -> . NP VP PP>
bottom-up: <0, 0, S -> . NP VP>
chart = Main.Charts.Edge[<0, 1, PN -> mia .>, <0, 0, NP -> . PN>, <0, 1, NP -> PN .>]
agenda = Main.Charts.Edge[<0, 0, S -> . NP VP>, <0, 0, S -> . NP VP PP>, <1, 2, IV -> danced .>]
stdin> 
moving: <0, 0, S -> . NP VP> to chart
fundamental rule: <0, 1, S -> NP . VP>
chart = Main.Charts.Edge[<0, 1, PN -> mia .>, <0, 0, NP -> . PN>, <0, 1, NP -> PN .>, <0, 0, S -> . NP VP>]
agenda = Main.Charts.Edge[<0, 1, S -> NP . VP>, <0, 0, S -> . NP VP PP>, <1, 2, IV -> danced .>]
stdin> 
moving: 